# Nhận dạng chữ số viết tay bằng Support Vector Machine

---
## Import thư viện cần thiết

In [1]:
import numpy as np
import pickle
import gzip

---
## Hàm đọc dữ liệu

In [2]:
def read_mnist(mnist_file):
    """
    Reads MNIST data.
    
    Parameters
    ----------
    mnist_file : string
        The name of the MNIST file (e.g., 'mnist.pkl.gz').
    
    Returns
    -------
    (train_X, train_Y, val_X, val_Y, test_X, test_Y) : tuple
        train_X : numpy array, shape (N=50000, d=784)
            Input vectors of the training set.
        train_Y: numpy array, shape (N=50000)
            Outputs of the training set.
        val_X : numpy array, shape (N=10000, d=784)
            Input vectors of the validation set.
        val_Y: numpy array, shape (N=10000)
            Outputs of the validation set.
        test_X : numpy array, shape (N=10000, d=784)
            Input vectors of the test set.
        test_Y: numpy array, shape (N=10000)
            Outputs of the test set.
    """
    f = gzip.open(mnist_file, 'rb')
    train_data, val_data, test_data = pickle.load(f, encoding='latin1')
    f.close()
    
    train_X, train_Y = train_data
    val_X, val_Y = val_data
    test_X, test_Y = test_data    
    
    return train_X, train_Y, val_X, val_Y, test_X, test_Y

In [3]:
# Test dữ liệu đã đọc
train_X, train_Y, val_X, val_Y, test_X, test_Y = read_mnist('mnist.pkl.gz')

print('train_X.shape =', train_X.shape)
print('train_Y.shape =', train_Y.shape)
print('val_X.shape   =', val_X.shape)
print('val_Y.shape   =', val_Y.shape)
print('test_X.shape  =', test_X.shape)
print('test_Y.shape  =', test_Y.shape)

print('\ntrain_X: min = %.3f, max = %.3f' %(train_X.min(), train_X.max()))
print('train_Y: min = %d, max = %d' %(train_Y.min(), train_Y.max()))

train_X.shape = (50000, 784)
train_Y.shape = (50000,)
val_X.shape   = (10000, 784)
val_Y.shape   = (10000,)
test_X.shape  = (10000, 784)
test_Y.shape  = (10000,)

train_X: min = 0.000, max = 0.996
train_Y: min = 0, max = 9


---
## Import thư viện SVM

In [4]:
from sklearn.svm import SVC
import time
from sklearn.externals import joblib # Để lưu lại model

---
## Hàm ghi file

In [5]:
def writeFileLinear(fileName, data, C):
    with open(fileName, 'w') as f:
        for i in range(len(data)):
            f.write(str(C[i]) + ' ' + str(data[i]) + '\n')
            
def writeFileRBF(fileName, data, C, gamma):
    k = 0
    with open(fileName, 'w') as f:
        for c in C:
            for g in gamma:
                f.write(str(c) + ' ' + str(g) + ' ' + str(data[k]) + '\n')
                k += 1

---
## Khởi tạo biến để lưu độ lỗi và thời gian của Linear Kernel

In [6]:
lossTrainLinear = [] # Độ lỗi dữ liệu trên tập train
lossValLinear = [] # Độ lỗi dữ liệu trên Validation
timeTrainLinear = [] # Thời gian chạy huấn luyện

---
## Chạy để train Linear Kernel

In [35]:
fileName = 'SVMLinearWithC_'
CTrainLinear = [0.001, 0.01, 0.05, 0.1, 1, 3, 5, 7, 10, 20, 30, 100, 300, 500, 1000]

for CTrain in CTrainLinear:
    model = SVC(kernel = 'linear', C = CTrain)
    start = time.time() # Set thời gian bắt đầu train
    model.fit(train_X, train_Y)
    timeTrainLinear.append(time.time() - start)
    lossTrainLinear.append(1.0 - model.score(train_X, train_Y))
    lossValLinear.append(1.0 - model.score(val_X, val_Y))
    joblib.dump(model, fileName + str(CTrain) + '.model') # Lưu lại model

---
## Chạy để ghi file Linear:

In [36]:
writeFileLinear('Linear_LossTrain.data', lossTrainLinear, CTrainLinear)
writeFileLinear('Linear_LossVal.data', lossValLinear, CTrainLinear)
writeFileLinear('Linear_TimeTrain.data', timeTrainLinear, CTrainLinear)

---
## Khởi tạo biến để lưu độ lỗi và thời gian của RBF

In [39]:
lossTrainRBF = [] # Độ lỗi dữ liệu trên tập train
lossValRBF = [] # Độ lỗi dữ liệu trên tập Validation
timeTrainRBF = [] # Thời gian chạy huấn luyện

---
## Chạy để train RBF:

In [46]:
fileName = 'SVMGaussianWithC_'

CTrainRBF = [0.1, 1, 5, 7, 10, 20, 30]
gammas = [0.001, 0.01, 0.1]
for CTrain in CTrainRBF:
    for g in gammas:
        model = SVC(kernel = 'rbf', gamma = g, C = CTrain)
        start = time.time()
        model.fit(train_X, train_Y)
        timeTrainRBF.append(time.time() - start)
        lossTrainRBF.append(1.0 - model.score(train_X, train_Y))
        lossValRBF.append(1.0 - model.score(val_X, val_Y))
        joblib.dump(model, fileName + str(CTrain) + '-Gamma_' + str(g) + '.model')

---
## Chạy để ghi file RBF:

In [52]:
writeFileRBF('RBF_LossTrain.data', lossTrainRBF, CTrainRBF, gammas)
writeFileRBF('RBF_LossVal.data', lossValRBF, CTrainRBF, gammas)
writeFileRBF('RBF_TimeTrain.data', timeTrainRBF, CTrainRBF, gammas)

---
## Chọn model có lossVal nhỏ nhất để test:

In [45]:
print('Độ lỗi nhỏ nhất trên tập validation: %f' % (np.min(lossValLinear + lossValRBF)))
index = np.argmin(lossValLinear + lossValRBF)
if index < len(lossValLinear):
    fileModel = 'SVMLinearWithC_' + str(CTrainLinear[index]) + '.model' #sua lai cai ten file cho dung
else:
    index = index - len(lossValLinear)
    fileModel = 'SVMGaussianWithC_' + str(CTrainRBF[index]) + '-Gamma_' + str(gammas[index]) + '.model' #sua lai cai ten file cho dung

model = joblib.load(fileModel)
print('Mô hình được chọn: ')
print(model)
print('Độ lỗi trên tập test: %f' % (1 - model.score(test_X, test_Y)))

Độ lỗi nhỏ nhất trên tập validation: 0.016500
Mô hình được chọn: 
SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Độ lỗi trên tập test: 0.018000
